In [1]:
import argparse
import dill
from datetime import datetime
import numpy as np
import pickle
import qiskit as qk
from qiskit.quantum_info import Operator, Statevector
from qiskit.circuit.quantumcircuit import QuantumCircuit
from typing import Tuple

from akash.TFIM_ham_gen import construct_hamiltonian

import dreamcoder as dc
from dreamcoder.frontier import Frontier, FrontierEntry
from dreamcoder.fragmentGrammar import FragmentGrammar
from dreamcoder.grammar import Grammar
from dreamcoder.program import Program
from dreamcoder.program import Abstraction
from dreamcoder.task import Task
from dreamcoder.utilities import numberOfCPUs
import dreamcoder.domains.quantum_ground_state.primitives as pr
from dreamcoder.domains.quantum_ground_state.primitives import (
    mat_contraction,
    mat_to_tensor,
    circuit_to_mat,
    tensor_contraction,
    execute_program,
    normalize_unitary,
    get_qiskit_circuit,
    get_instructions_from_qiskit,
    get_code_from_instructions,
    tcircuit,
    no_op,
    QiskitTester,
)
from dreamcoder.domains.quantum_ground_state.primitives import execute_quantum_algorithm
from dreamcoder.domains.quantum_ground_state.tasks import GroundStateTask,get_energy
from dreamcoder.program import Program
from dreamcoder.utilities import eprint

%load_ext autoreload
%autoreload 2

In [2]:
mag_field_strength_list = [0.001, 0.1, 1, 1.5]
decomposed_list = [0, 1]


class args:
    n_qubits = 2
    J = 1
    hh = mag_field_strength_list[0]
    decomposed = 1
    arity = 2
    structurePenalty = 1
    pseudoCounts = 10


# Read the command line arguments
parser = argparse.ArgumentParser(
    description="Example implementation of Regularized Mutual Information Feature Selector on a solid drop.",
    epilog="Results will be saved in files with the OUTPUT tag in the 'outputs/' folder.",
    formatter_class=argparse.ArgumentDefaultsHelpFormatter,
)

parser.add_argument(
    "-n_qubits", type=int, default=args.n_qubits, help="Number of qubits"
)
parser.add_argument("-J", type=float, default=args.J, help="Interaction strength")
parser.add_argument("-hh", type=float, default=args.hh, help="External field strength")
parser.add_argument(
    "-decomposed",
    type=int,
    default=args.decomposed,
    help="Either 0=parametrized gates, 1=qiskit hardware basis",
)
parser.add_argument(
    "-arity",
    type=int,
    default=args.arity,
    help="Number of arguments of extracted gates",
)
parser.add_argument(
    "-structurePenalty", type=int, default=args.structurePenalty, help="hyperparameter"
)
parser.add_argument(
    "-pseudoCounts", type=int, default=args.pseudoCounts, help="hyperparameter"
)

try:
    args = parser.parse_args()
except SystemExit as e:
    eprint("Running from interactive session. Loading default parameters")

usage: ipykernel_launcher.py [-h] [-n_qubits N_QUBITS] [-J J] [-hh HH]
                             [-decomposed DECOMPOSED] [-arity ARITY]
                             [-structurePenalty STRUCTUREPENALTY]
                             [-pseudoCounts PSEUDOCOUNTS]
ipykernel_launcher.py: error: unrecognized arguments: --f=/mnt/home/lsarra/.local/share/jupyter/runtime/kernel-v2-1268611m1P5WeL3Q5eq.json
313540504.py:49 > Running from interactive session. Loading default parameters


In [6]:
path = f"akash/solved_RL_circuits/circ_list_TFIM_qubit{args.n_qubits}_J{args.J}_h{args.hh}_decomposed{args.decomposed}.pickle"
name = f"ground_{args.n_qubits}_J{args.J}_h{args.hh}_dec{args.decomposed}"
with open(path, "rb") as handle:
    b = dill.load(handle)
eprint(f"Loading solutions from {path}")

4151542144.py:5 > Loading solutions from akash/solved_RL_circuits/circ_list_TFIM_qubit2_J1_h0.001_decomposed1.pickle


In [7]:
# Unfortunately these flags are set globally
dc.domains.quantum_ground_state.primitives.GLOBAL_NQUBIT_TASK = args.n_qubits
dc.domains.quantum_ground_state.primitives.GLOBAL_LIMITED_CONNECTIVITY = False

library_settings = {
    "topK": 2,  # how many solutions to consider
    "arity": args.arity,  # how many arguments
    "structurePenalty": args.structurePenalty,  # increase regularization 3 4 (it was 1), look at a few in [1,15]
    "pseudoCounts": args.pseudoCounts,  # increase to 100, test a few values
}

primitives = [pr.p_sx, pr.p_x, pr.p_rz, pr.p_cz]
grammar = Grammar.uniform(primitives)
eprint(f"Library building settings: {library_settings}")

2954477143.py:14 > Library building settings: {'topK': 2, 'arity': 2, 'structurePenalty': 1, 'pseudoCounts': 10}


In [9]:
# Generate a few example tasks
solutions = {}  # dict of task:solution
# NOTE: we have a task for each decomposition because they have various different real parameters
# We cannot have solutions with different requests for a task,
# and it is not clear how to use real numbers as primitives (just for evaluation, we cannot enumerate them)
for idx,circuit in enumerate(b):
    H = construct_hamiltonian(args.J, args.hh, args.n_qubits)
    instructions = get_instructions_from_qiskit(circuit)
    code, arguments = get_code_from_instructions(instructions)
    program = Program.parse(code)
    task = GroundStateTask(f"J_{args.J:2.2f}_h_{args.hh:2.2f}_N_{args.n_qubits}_v{idx}",hamiltonian=H, arguments=arguments, request = program.infer())
    likelihood = task.logLikelihood(program)
    prior = grammar.logLikelihood(program.infer(), program)

    frontier_entry = FrontierEntry(
        program=program, logLikelihood=likelihood, logPrior=prior
    )

    solutions[task] = Frontier(
        frontier=[frontier_entry],  # multiple solutions are allowed
        task=task,
    )
    eprint(f"#{idx:3}, Energy = {likelihood:2.6f}")
tasks = list(solutions.keys())

241499910.py:23 > #  0, Energy = 1.000002
241499910.py:23 > #  1, Energy = 1.000000
241499910.py:23 > #  2, Energy = 1.000000
241499910.py:23 > #  3, Energy = 1.000000
241499910.py:23 > #  4, Energy = 1.000000
241499910.py:23 > #  5, Energy = 1.000000
241499910.py:23 > #  6, Energy = 1.000000
241499910.py:23 > #  7, Energy = 1.000000
241499910.py:23 > #  8, Energy = 1.000000
241499910.py:23 > #  9, Energy = 1.000000


241499910.py:23 > # 10, Energy = 1.000000
241499910.py:23 > # 11, Energy = 1.000000
241499910.py:23 > # 12, Energy = 1.000000
241499910.py:23 > # 13, Energy = 1.000000
241499910.py:23 > # 14, Energy = 1.000000
241499910.py:23 > # 15, Energy = 1.000000
241499910.py:23 > # 16, Energy = 1.000000
241499910.py:23 > # 17, Energy = 1.000000
241499910.py:23 > # 18, Energy = 1.000000
241499910.py:23 > # 19, Energy = 1.000000


In [10]:
frontiers = [f for f in solutions.values()]

unique_frontiers_set = set()
unique_frontiers = []
for frontier in frontiers:
    program=  frontier.entries[0].program
    if program not in unique_frontiers_set:
        unique_frontiers_set.add(program)
        unique_frontiers.append(frontier)
eprint(f"We have {len(unique_frontiers)}/{len(frontiers)} frontiers. The others are duplicate solutions")

unique_frontiers

2373419330.py:10 > We have 8/20 frontiers. The others are duplicate solutions


[Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (sx (x (sx (cz (sx (rz (cz (rz (x (sx (rz (sx $0 $2) $5 $1) $1) $2) $4 $2) $1 $2) $3 $2) $2) $1 $2) $1) $2) $2))))))), logPrior=-33.92259025548687, logLikelihood=1.0000019905250335], task=J_1.00_h_0.00_N_2_v0),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (sx (cz (x (x (sx (sx (rz (x (rz (x (rz (sx (cz (x (rz (sx (cz (rz (cz (rz (x (cz (sx (rz (x $0 $2) $9 $1) $2) $1 $2) $2) $8 $1) $1 $2) $7 $2) $1 $2) $2) $6 $1) $1) $1 $2) $1) $5 $2) $2) $4 $1) $1) $3 $2) $1) $2) $1) $2) $1 $2) $2))))))))))), logPrior=-76.26117218347213, logLikelihood=1.0000003891172966], task=J_1.00_h_0.00_N_2_v1),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (x (sx (x (sx (x (rz (sx (rz (x (sx (sx (cz (x (cz (sx $0 $1) $1 $2) $1) $1 $2) $1) $2) $1) $4 $2) $2) $3 $1) $1) $1) $1) $1) $1)))))), logPrior=-38.92080302958456

In [11]:
new_grammar, new_frontiers = FragmentGrammar.induceFromFrontiers(
    g0=grammar, frontiers=unique_frontiers[:2], **library_settings,
    CPUs=numberOfCPUs()
)

new_grammar, new_frontiers

fragmentGrammar.py:298 > Inducing a grammar from 2 frontiers


fragmentGrammar.py:326 > Starting score -120.48116418744559
fragmentGrammar.py:333 > Proposed 137 fragments.
fragmentGrammar.py:364 > New primitive of type tcircuit	(rz (cz (rz (x $0 $1) $2 $3) $4 $1) $5 $1)	
(score = -116.353017; dScore = 4.128147; <uses> = 1.998121)
fragmentGrammar.py:377 > 	(<uses> in rewritten frontiers: 2.000000)
fragmentGrammar.py:333 > Proposed 68 fragments.
fragmentGrammar.py:364 > New primitive of type int -> tcircuit	(sx (cz $0 $1 $2))	
(score = -116.241393; dScore = 0.111624; <uses> = 3.552372)
fragmentGrammar.py:377 > 	(<uses> in rewritten frontiers: 4.000000)
fragmentGrammar.py:333 > Proposed 36 fragments.
fragmentGrammar.py:396 > Old joint = -108.183760	New joint = -99.829394

fragmentGrammar.py:416 > 7.000000 / 29.000000	sx
fragmentGrammar.py:416 > 7.000000 / 29.000000	x
fragmentGrammar.py:416 > 6.000000 / 29.000000	rz
fragmentGrammar.py:416 > 1.000000 / 29.000000	cz
fragmentGrammar.py:416 > 2.000000 / 29.000000	#(lambda (lambda (lambda (lambda (lambda (

(<dreamcoder.grammar.Grammar at 0x7f2a56d73cd0>,
 [Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (sx (x (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (sx (#(lambda (lambda (lambda (lambda (lambda (lambda (rz (cz (rz (x $0 $1) $2 $3) $4 $1) $5 $1))))))) $3 $1 $2 $4 $2 (sx (rz (sx $0 $2) $5 $1) $1)) $2) $1) $2) $2))))))), logPrior=-29.003119784129332, logLikelihood=1.0000019905250335], task=J_1.00_h_0.00_N_2_v0),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (x (x (sx (sx (rz (x (rz (x (rz (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (x (rz (#(lambda (lambda (lambda (sx (cz $0 $1 $2))))) $2 $1 (#(lambda (lambda (lambda (lambda (lambda (lambda (rz (cz (rz (x $0 $1) $2 $3) $4 $1) $5 $1))))))) $7 $1 $1 $8 $2 (cz (sx (rz (x $0 $2) $9 $1) $2) $1 $2)) $2) $6 $1) $1) $1) $5 $2) $2) $4 $1) $1) $3 $2) $1) $2) $

In [7]:
timestamp = datetime.now().isoformat()
with open(f"experimentOutputs/{timestamp}_{name}_grammar.pickle","wb") as f:
    pickle.dump(new_grammar,f)
    
with open(f"experimentOutputs/{timestamp}_{name}_frontiers.pickle","wb") as f:
    pickle.dump(new_frontiers,f)
eprint(f"Results saved in experimentOutputs/{timestamp}_{name}_...")

---

## Some analysis

(cz ($0 (cz ($0 (cz ($0 (cz ($0 (cz ($0 (cz ($0 (cz ($0 (x ($0 (x ($0 (x ($0 (x (cz ($0 ($0 (cz ($0 ($0 (x (x $1 $2) $3) $4 $2) $5 $3) $2 $3) $6 $2) $7 $3) $2 $3) $2) $8 $3) $3) $9 $2) $2) $10 $3) $3) $11 $2) $2 $3) $12 $2) $2 $3) $13 $3) $2 $3) $14 $2) $2 $3) $15 $3) $2 $3) $16 $3) $2 $3) $17 $2) $2 $3)	


In [33]:
frontiers

[Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (sx (x (sx (cz (sx (rz (cz (rz (x (sx (rz (sx $0 $2) $5 $1) $1) $2) $4 $2) $1 $2) $3 $2) $2) $1 $2) $1) $2) $2))))))), logPrior=-33.92259025548687, logLikelihood=1.0000019905250335], task=J_1.00_h_0.00_N_2_v0),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (lambda (sx (cz (x (x (sx (sx (rz (x (rz (x (rz (sx (cz (x (rz (sx (cz (rz (cz (rz (x (cz (sx (rz (x $0 $2) $9 $1) $2) $1 $2) $2) $8 $1) $1 $2) $7 $2) $1 $2) $2) $6 $1) $1) $1 $2) $1) $5 $2) $2) $4 $1) $1) $3 $2) $1) $2) $1) $2) $1 $2) $2))))))))))), logPrior=-76.26117218347213, logLikelihood=1.0000019905250335], task=J_1.00_h_0.00_N_2_v1),
 Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (lambda (x (sx (x (sx (x (rz (sx (rz (x (sx (sx (cz (x (cz (sx $0 $1) $1 $2) $1) $1 $2) $1) $2) $1) $4 $2) $2) $3 $1) $1) $1) $1) $1) $1)))))), logPrior=-38.92080302958456

In [163]:
instructions = get_instructions_from_qiskit(circuit)
code, arguments = get_code_from_instructions(instructions)
program = Program.parse(code)
reconstructed = execute_program(program, arguments)

# Test

In [ ]:
# psi0 = np.array([1,0,0,0])
# rot = np.array(Operator(circ).data)
# psi1= np.dot(rot,psi0)

In [ ]:
# pipeline is
# CODE:str --> PROGRAM:Program --> INSTRUCTIONS:tuple --> CIRCUIT: QuantumCircuit

# inverse pipeline is
# CIRCUIT: QuantumCircuit --> INSTRUCTIONS: tuple --> CODE: str 

In [ ]:
H = construct_hamiltonian(J, h, n_qubits)
for circ in b:
    psi0 = Statevector.from_int(0, 2**n_qubits)
    psi1 = psi0.evolve(circ)
    print(get_energy(psi1.data, H))
    
instructions = get_instructions_from_qiskit(circ)
code, arguments = get_code_from_instructions(instructions)
program = Program.parse(code)

(-1.0000019905250335+0j)
(-1.000000389117297-5.551115123125783e-17j)
(-1.0000001942192795+0j)
(-1.0000000807637257+0j)
(-1.0000000793832347-5.551115123125783e-17j)
(-1.0000000000000002+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-1.0000000000000004+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)
(-0.9999999999999998+0j)


In [ ]:
# Test: get_qiskit_circuit(instructions) --> same circuit
instructions = get_instructions_from_qiskit(circ)
reconstructed_circuit = get_qiskit_circuit(instructions)
Operator(reconstructed_circuit.circuit).data - Operator(circ).data

In [ ]:
# Test: get code from instructions
code, arguments = get_code_from_instructions(instructions)
program = Program.parse(code)
reconstructed = execute_program(program, arguments)
reconstructed == instructions

print(program)
print(reconstructed)
print(instructions)


In [16]:
primitives

[sx, x, rz, cz]

In [21]:
# Test: unitary from instructions
with QiskitTester(2) as QT:
    QT.circuit.rz(1,1)
QT.circuit.draw()
instructions = get_instructions_from_qiskit(QT.circuit)
Operator(QT.circuit).data, circuit_to_mat(instructions)

(array([[0.87758256-0.47942554j, 0.        +0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.87758256-0.47942554j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.87758256+0.47942554j, 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.        +0.j        , 0.87758256+0.47942554j]]),
 array([[0.87758256-0.47942554j, 0.        +0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.87758256-0.47942554j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.87758256+0.47942554j, 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.        +0.j        , 0.87758256+0.47942554j]]))

In [ ]:
instructions = (2, (
    ("sxdg", 0),
    ("x", 0),
    ("x", 1),
    ("rz", 1.43,1),
    ("cnot", 0,1)
    ))
get_qiskit_circuit(instructions).circuit.draw()
# with QiskitTester(circuit=instructions) as QT:
#     QT.circuit.sxdg(QT.q(0))
# QT.circuit.draw()

┌───┐  ┌──────────┐┌───┐
q_0: ─┤ X ├──┤ Rz(1.43) ├┤ X ├
     ┌┴───┴─┐└──┬───┬───┘└─┬─┘
q_1: ┤ √Xdg ├───┤ X ├──────■──
     └──────┘   └───┘

In [ ]:
instructions = (1, (
    # ("sxdg", 0),
    # ("x", 0),
    # ("x", 1),
    ("rz", 1.43,0),
    # ("cnot", 0,1)
    ))
get_qiskit_circuit(instructions).circuit.draw()
code, arguments = get_code_from_instructions(instructions)

In [ ]:
code = '(lambda (lambda(lambda (rz $0 $2 $1 ) )))'
code = '(cnot (no_op(2)) 1 0)'
code = '(cnot (I 2) 0 1)'
code = '(x (I 1) 0 )'
code = '(x (rz (I 1) 11 0) 0 )'

In [ ]:
program = Program.parse(code)
instructions = execute_program(program, ())
get_qiskit_circuit(instructions).circuit.draw()

Exception: Attempt to evaluate hole

In [ ]:
for circ in b:
    H = construct_hamiltonian(J, h, n_qubits)
    instructions = get_instructions_from_qiskit(circ)
    code, arguments = get_code_from_instructions(instructions)
    program = Program.parse(code)
    task = GroundStateTask(f"J_{J:2.2f}_h_{h:2.2f}_N_{n_qubits}",hamiltonian=H, arguments=arguments, request = program.infer())
    energy = task.logLikelihood(program)
    
    psi0 = Statevector.from_int(0, 4)
    psi1 = psi0.evolve(circ)
    print(get_energy(psi1,H),energy)

(-1.0000019905250335+0j) 1.0000019905250335
(-1.000000389117297-5.551115123125783e-17j) 1.0000003891172966
(-1.0000001942192795+0j) 1.0000001942192795
(-1.0000000807637257+0j) 1.0000000807637257
(-1.0000000793832347-5.551115123125783e-17j) 1.000000079383235
(-1.0000000000000002+0j) 1.000000000000001
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-1.0000000000000004+0j) 1.0000000000000009
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007
(-0.9999999999999998+0j) 1.0000000000000007


In [ ]:
instructions

(2,
 (('x', 0),
  ('rz', 12.852044105529785, 1),
  ('sx', 0),
  ('rz', 4.629762649536133, 0),
  ('cz', 1, 0),
  ('rz', 5.277709484100342, 1),
  ('cz', 1, 0),
  ('sx', 0),
  ('rz', 0.030434206128120422, 1),
  ('x', 1),
  ('x', 0),
  ('rz', -1.6001750230789185, 1),
  ('x', 1),
  ('cz', 1, 0),
  ('x', 1),
  ('x', 0),
  ('cz', 1, 0),
  ('rz', -0.9125978946685791, 1),
  ('cz', 1, 0),
  ('sx', 0),
  ('x', 1),
  ('rz', -1.3367031812667847, 1),
  ('rz', -1.1753524541854858, 0),
  ('x', 1),
  ('cz', 1, 0),
  ('sx', 1),
  ('x', 0),
  ('cz', 1, 0),
  ('sx', 1)))